In [12]:
# パッケージをロード
library(tidyverse)
library(sf)
library(pforeach) #githubinstall::githubinstall("pforeach")

# シェイプファイルから検索するやり方
ベンチマークなので本来は1万回とか繰り返して実行時間の分布をみるべきですが、時間がかかるので1回だけの実行時間で見ています。

また、Pythonでは全国シェイプファイルから直接検索していますがRではそれだと遅すぎるので少しでも速くするために「都道府県を検索→都道府県の中で番地検索」というアプローチにしています。

In [2]:
# 桁数の設定
options(digits = 15)

In [3]:
# 関数定義
#' @param polygon list
#' @param lon longitude
#' @param lat latitude
## 都道府県の判定
find_pref2 <- function(polygon, lon, lat){
  point <- st_point(c(lon, lat))
  for(p in pref){
    if(length(st_contains(polygon[[p]], point)[[1]])){
      return(p)
    }
  }
  return(NA)
}

## 住所(何丁目レベル)判定
#' @param sp_polygon object of class sf, sfc or sfg
#' @param pref prefecture
#' @param lon longitude
#' @param lat latitude
find_place <- function(sp_polygon, pref, lon, lat) {

  if(is.na(pref) == TRUE) {
    return(NA)
  }

  sp_pref <- sp_polygon %>%
    dplyr::mutate_if(is.factor, as.character) %>%
    filter(KEN_NAME == pref)

  which.row <- suppressMessages(sf::st_contains(sp_pref, sf::st_point(c(lon, lat)), sparse = FALSE)) %>%
    grep(TRUE, .)

  if (identical(which.row, integer(0)) == TRUE) {
    return(NA)
  } else {
    geos <- sp_pref[which.row, ]

    place_name = paste(geos$GST_NAME,
                       geos$CSS_NAME,
                       geos$MOJI,
                       sep = ",") %>%
      stringr::str_replace_all("NA", "")
    return(place_name)
  }
}

## 県と市区町村、番地を検索
##' @param polygon polygon list
##' @param sp 番地レベルshapefile
##' @param longitude 経度
##' @param latitude 緯度
find_pref_place <- function(polygon, sp2, longitude, latitude) {
  pref <- find_pref2(polygon, longitude, latitude)
  if(is.na(pref) == TRUE) {
    return(NA)
  }
  city_banchi <- find_place(sp2, pref, longitude, latitude)
  return(paste(pref, city_banchi, sep = ","))
}

In [4]:
# 都道府県のshapefile読み込み
load("./R/jp.RData")
japan_p <- select(jp, KEN)
pref <- unique(jp$KEN)
# 都道府県ごとにポリゴンをまとめる
pref_polygons <- tapply(japan_p$geometry,
                        japan_p$KEN, # 都道府県名でグループ化
                        st_combine)

In [5]:
# 市区町村レベルのshapefileの読み込み
load("./R/zenkoku.RData")

## 1件のみ逆ジオコーディング

In [6]:
find_pref_place(pref_polygons, zenkoku, 135.728965, 35.039200)

[1] "京都府,京都市,北区,金閣金閣寺町"

In [7]:
find_pref_place(pref_polygons, zenkoku, 139.523620, 35.495978)

[1] "神奈川県,横浜市,旭区,上白根町"

In [8]:
# 実行時間
system.time(
find_pref_place(pref_polygons, zenkoku, 139.523620, 35.495978)
)

            user           system          elapsed 
2.49300000000000 0.01700000000000 2.52100000000007 

2.5秒

## 1000件の逆ジオコーディング

In [10]:
# データの読み込み
# 4箇所を250回繰り返しただけ
dat <- read_csv("./dat1000.csv")
dat

Parsed with column specification:
cols(
  long = col_double(),
  lat = col_double(),
  location = col_character(),
  id = col_integer()
)


long,lat,location,id
139.884678,35.626065,東京ディズニーシー,1
139.742946,35.649114,慶應義塾大学,2
135.728965,35.039200,金閣寺,3
139.526510,35.494865,よこはま動物園ズーラシア,4
139.884678,35.626065,東京ディズニーシー,5
139.742946,35.649114,慶應義塾大学,6
135.728965,35.039200,金閣寺,7
139.526510,35.494865,よこはま動物園ズーラシア,8
139.884678,35.626065,東京ディズニーシー,9
139.742946,35.649114,慶應義塾大学,10


### 並列処理で計算
データをコア数分に分割し処理を分ける

In [19]:
# 実行
res <- pforeach(row = rows(dat), .c = rbind)({
  row %>%
    mutate(place = find_pref_place(pref_polygons, zenkoku, long, lat))
})

In [21]:
res

long,lat,location,id,place
139.884678,35.626065,東京ディズニーシー,1,"千葉県,浦安市,,舞浜"
139.742946,35.649114,慶應義塾大学,2,"東京都,港区,,三田２丁目"
135.728965,35.039200,金閣寺,3,"京都府,京都市,北区,金閣金閣寺町"
139.526510,35.494865,よこはま動物園ズーラシア,4,"神奈川県,横浜市,旭区,上白根町"
139.884678,35.626065,東京ディズニーシー,5,"千葉県,浦安市,,舞浜"
139.742946,35.649114,慶應義塾大学,6,"東京都,港区,,三田２丁目"
135.728965,35.039200,金閣寺,7,"京都府,京都市,北区,金閣金閣寺町"
139.526510,35.494865,よこはま動物園ズーラシア,8,"神奈川県,横浜市,旭区,上白根町"
139.884678,35.626065,東京ディズニーシー,9,"千葉県,浦安市,,舞浜"
139.742946,35.649114,慶應義塾大学,10,"東京都,港区,,三田２丁目"


In [26]:
# 列を分割
res2 <- res %>%
  separate(place, c("pref", "city", "town", "banchi"), sep = ",")
res2[res2 == ""] <- NA
res2[res2 == "NA"] <- NA
res2

long,lat,location,id,pref,city,town,banchi
139.884678,35.626065,東京ディズニーシー,1,千葉県,浦安市,NA,舞浜
139.742946,35.649114,慶應義塾大学,2,東京都,港区,NA,三田２丁目
135.728965,35.039200,金閣寺,3,京都府,京都市,北区,金閣金閣寺町
139.526510,35.494865,よこはま動物園ズーラシア,4,神奈川県,横浜市,旭区,上白根町
139.884678,35.626065,東京ディズニーシー,5,千葉県,浦安市,NA,舞浜
139.742946,35.649114,慶應義塾大学,6,東京都,港区,NA,三田２丁目
135.728965,35.039200,金閣寺,7,京都府,京都市,北区,金閣金閣寺町
139.526510,35.494865,よこはま動物園ズーラシア,8,神奈川県,横浜市,旭区,上白根町
139.884678,35.626065,東京ディズニーシー,9,千葉県,浦安市,NA,舞浜
139.742946,35.649114,慶應義塾大学,10,東京都,港区,NA,三田２丁目


In [27]:
# 実行時間
system.time(
  res <- pforeach(row = rows(dat), .c = rbind)({
  row %>%
    mutate(place = find_pref_place(pref_polygons, zenkoku, long, lat))
})
)

    user   system  elapsed 
6993.770  166.467 2104.640 

約35分...